# Symetrical log transform

In [1]:
import numpy as np
import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
DEF_BASE = 10

In [4]:
def symlog_trans(x, base, threshold, scale):
    # https://gist.github.com/davidtedfordholt/3e6ed189c58af941069991aef79b45a9
    import warnings

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        return np.where(
            np.abs(x) < threshold,
            x,
            np.sign(x) * (threshold + scale * np.emath.logn(base, np.sign(x) * x / threshold))
        )

def matlab_symlog(x, c=-1.):
    # https://www.mathworks.com/matlabcentral/fileexchange/57902-symlog
    return np.sign(x) * np.emath.logn(10., 1. + np.abs(x) / 10.**c)

def get_data(target_values, base=DEF_BASE, threshold=1., scale=1.):
    from matplotlib.scale import SymmetricalLogTransform

    return pd.DataFrame({
        'values': np.random.normal(size=len(target_values)),
        'target': target_values,
        'matplotlib_symlog': SymmetricalLogTransform(base=base, linthresh=threshold, linscale=scale).transform(target_values),
        'ggplot2_symlog': symlog_trans(target_values, base, threshold, scale),
        'matlab_symlog': matlab_symlog(target_values),
    })

def get_non_negative_target_values(n=100, base=DEF_BASE, max_deg=10, seed=42):
    np.random.seed(seed)
    return [base**d if d >= 0 else 0 for d in np.random.randint(max_deg + 2, size=n) - 1]

def get_all_target_values(n=100, base=DEF_BASE, max_deg=10, seed=42):
    np.random.seed(seed)
    return [base**d if d >= 0 else -base**(-d) for d in np.random.randint(max_deg * 2 + 1, size=n) - max_deg]

def get_sin_data(n=2_000, start=-50, stop=50, base=DEF_BASE, threshold=1., scale=1.):
    from matplotlib.scale import SymmetricalLogTransform

    x = np.linspace(start, stop, n)
    y = np.sin(x)
    return pd.DataFrame({
        'x': x,
        'y': y,
        'matplotlib_symlog': SymmetricalLogTransform(base=base, linthresh=threshold, linscale=scale).transform(x),
        'ggplot2_symlog': symlog_trans(x, base, threshold, scale),
        'matlab_symlog': matlab_symlog(x),
    })

In [5]:
df1 = get_data(get_non_negative_target_values())
print(df1.shape)
df1.head()

(100, 5)


,values,target,matplotlib_symlog,ggplot2_symlog,matlab_symlog
0,-0.043537,100000,6.111111,6.0,6.000000
1,-0.275301,100,3.111111,3.0,3.000434
2,-1.563067,1000000000,10.111111,10.0,10.000000
3,-0.795631,1000000,7.111111,7.0,7.000000
4,0.930584,1000,4.111111,4.0,4.000043


In [6]:
df2 = get_data(get_all_target_values())
print(df2.shape)
df2.head()

(100, 5)


,values,target,matplotlib_symlog,ggplot2_symlog,matlab_symlog
0,-0.896335,-10000,-5.111111,-5.0,-5.000004
1,-0.111988,1000000000,10.111111,10.0,10.000000
2,1.468941,10000,5.111111,5.0,5.000004
3,-1.123898,1,1.111111,1.0,1.041393
4,0.950005,-1000,-4.111111,-4.0,-4.000043


In [7]:
df3 = get_sin_data()
print(df3.shape)
df3.head()

(2000, 5)


,x,y,matplotlib_symlog,ggplot2_symlog,matlab_symlog
0,-50.000000,0.262375,-2.810081,-2.698970,-2.699838
1,-49.949975,0.310299,-2.809646,-2.698535,-2.699404
2,-49.899950,0.357447,-2.809211,-2.698100,-2.698970
3,-49.849925,0.403700,-2.808776,-2.697665,-2.698535
4,-49.799900,0.448943,-2.808340,-2.697228,-2.698100


### Zero value in data

In [8]:
def p1(x):
    return ggplot(df1, aes(x, 'values', color=x)) + geom_point(show_legend=False)

gggrid([
    p1('target') + ggtitle("Default"),
    p1('target') + scale_x_log10() + ggtitle("log10"),
    p1('matplotlib_symlog') + xlab('symlog(target)') + ggtitle("symlog from matplotlib"),
    p1('ggplot2_symlog') + xlab('symlog(target)') + ggtitle("symlog from ggplot2"),
    p1('matlab_symlog') + xlab('symlog(target)') + ggtitle("symlog from matlab"),
], ncol=3)

### Negative values in data

In [9]:
def p2(x):
    return ggplot(df2, aes(x, 'values', color=x)) + geom_point(show_legend=False)

gggrid([
    p2('target') + ggtitle("Default"),
    p2('target') + scale_x_log10() + ggtitle("log10"),
    p2('matplotlib_symlog') + xlab('symlog(target)') + ggtitle("symlog from matplotlib"),
    p2('ggplot2_symlog') + xlab('symlog(target)') + ggtitle("symlog from ggplot2"),
    p2('matlab_symlog') + xlab('symlog(target)') + ggtitle("symlog from matlab"),
], ncol=3)

### Sin function

In [10]:
def p3(x):
    return ggplot(df3, aes(x, 'y', color=x)) + geom_line(show_legend=False)

gggrid([
    p3('x') + ggtitle("Default"),
    p3('x') + scale_x_log10() + ggtitle("log10"),
    p3('matplotlib_symlog') + xlab('symlog(target)') + ggtitle("symlog from matplotlib"),
    p3('ggplot2_symlog') + xlab('symlog(target)') + ggtitle("symlog from ggplot2"),
    p3('matlab_symlog') + xlab('symlog(target)') + ggtitle("symlog from matlab"),
], ncol=1)